# Data Cleaning & Exploratory Data Analysis

**Project:** Urban Mobility Optimization: A Data-Driven Framework for Sustainable Transportation Investment

**Objectives:**
1. Load and validate the raw dataset
2. Handle missing values strategically
3. Detect and treat outliers
4. Perform exploratory data analysis
5. Identify key patterns and relationships
6. Save cleaned dataset for modeling

---

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

print("Libraries imported successfully")

## 1. Load Data

In [ ]:
# Load raw data
df = pd.read_csv('../data/raw/world_bank_transport_data_raw.csv')

print("Dataset loaded successfully")
print(f"Shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")

In [ ]:
# First look at the data
df.head(10)

In [ ]:
# Data types
print("Data Types:")
print(df.dtypes)

In [ ]:
# Basic statistics
df.describe()

## 2. Missing Data Analysis

Understanding missingness patterns is critical for appropriate imputation strategies.

In [ ]:
# Missing data summary
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percent': (df.isnull().sum() / len(df) * 100).round(2)
})

missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Percent', ascending=False)

print("MISSING DATA ANALYSIS")
print("="*60)
print(missing_data.to_string(index=False))
print(f"\nOverall missingness: {(df.isnull().sum().sum() / (len(df) * len(df.columns)) * 100):.2f}%")

In [ ]:
# Visualize missing data
fig, ax = plt.subplots(figsize=(12, 6))

missing_cols = missing_data['Column'].tolist()
missing_pct = missing_data['Missing_Percent'].tolist()

bars = ax.barh(missing_cols, missing_pct, color='coral')
ax.set_xlabel('Missing Percentage (%)', fontsize=12)
ax.set_title('Missing Data by Indicator', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

# Add value labels
for i, (col, pct) in enumerate(zip(missing_cols, missing_pct)):
    ax.text(pct + 1, i, f'{pct:.1f}%', va='center')

plt.tight_layout()
plt.savefig('../visualizations/missing_data_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("Visualization saved: visualizations/missing_data_analysis.png")

### Missing Data Strategy

**LPI Scores (61% missing):** 
- By design - only available for specific years (2012, 2014, 2016, 2018, 2020)
- Strategy: Forward/backward fill within economy, then median imputation by income group

**Infrastructure Indicators (29% missing):**
- Rail lines: Not all economies have rail infrastructure
- Container traffic: Landlocked economies have no ports
- Strategy: Keep as NaN for economies without infrastructure, impute for data gaps

**Economic Indicators (7-9% missing):**
- Random missingness pattern
- Strategy: Linear interpolation within economy, then group median

In [ ]:
# Missingness by income group
print("Missing Data Patterns by Income Group:")
print("="*60)

for col in ['lpi_overall_score', 'rail_lines_total_km', 'gdp_per_capita_ppp']:
    if col in df.columns:
        print(f"\n{col}:")
        print(df.groupby('income_group')[col].apply(lambda x: f"{(x.isnull().sum() / len(x) * 100):.1f}%"))

## 3. Data Cleaning & Imputation

In [ ]:
# Create a copy for cleaning
df_clean = df.copy()

print(f"Starting cleaning process...")
print(f"Initial shape: {df_clean.shape}")
print(f"Initial missing values: {df_clean.isnull().sum().sum()}")

In [ ]:
# Strategy 1: Economic indicators - Interpolation within economy, then group median
economic_cols = ['gdp_per_capita_ppp', 'trade_pct_gdp', 'urban_population_pct', 
                 'co2_emissions_per_capita', 'energy_use_per_gdp', 
                 'gross_capital_formation_pct_gdp', 'co2_transport_pct_total',
                 'road_density_km_per_100sqkm', 'air_transport_passengers']

for col in economic_cols:
    if col in df_clean.columns:
        # Interpolate within each economy's time series
        df_clean[col] = df_clean.groupby('economy')[col].transform(
            lambda x: x.interpolate(method='linear', limit_direction='both')
        )
        
        # Fill remaining with income group median
        df_clean[col] = df_clean.groupby('income_group')[col].transform(
            lambda x: x.fillna(x.median())
        )

print("✓ Economic indicators imputed")

In [ ]:
# Strategy 2: LPI scores - Forward/backward fill, then income group median
lpi_cols = ['lpi_overall_score', 'lpi_infrastructure_score']

for col in lpi_cols:
    if col in df_clean.columns:
        # Forward fill then backward fill within economy
        df_clean[col] = df_clean.groupby('economy')[col].transform(
            lambda x: x.fillna(method='ffill').fillna(method='bfill')
        )
        
        # Fill remaining with income group median
        df_clean[col] = df_clean.groupby('income_group')[col].transform(
            lambda x: x.fillna(x.median())
        )

print("✓ LPI scores imputed")

In [ ]:
# Strategy 3: Infrastructure - Keep structural zeros, impute data gaps
# For rail and container traffic, some economies legitimately don't have these
# We'll create binary flags and fill missing with 0 for modeling purposes

df_clean['has_rail'] = df_clean['rail_lines_total_km'].notna().astype(int)
df_clean['has_port'] = df_clean['container_port_traffic_teu'].notna().astype(int)

# Fill with 0 (no infrastructure)
df_clean['rail_lines_total_km'] = df_clean['rail_lines_total_km'].fillna(0)
df_clean['container_port_traffic_teu'] = df_clean['container_port_traffic_teu'].fillna(0)

print("✓ Infrastructure indicators handled")
print(f"  Economies with rail: {df_clean.groupby('economy')['has_rail'].max().sum()}")
print(f"  Economies with ports: {df_clean.groupby('economy')['has_port'].max().sum()}")

In [ ]:
# Final missing data check
print("\nFinal Missing Data Check:")
print("="*60)
remaining_missing = df_clean.isnull().sum()
remaining_missing = remaining_missing[remaining_missing > 0]

if len(remaining_missing) > 0:
    print(remaining_missing)
else:
    print("✓ No missing values remaining!")

print(f"\nCleaned dataset shape: {df_clean.shape}")

## 4. Outlier Detection & Treatment

In [ ]:
# Function to detect outliers using IQR method
def detect_outliers_iqr(df, column, multiplier=1.5):
    """
    Detect outliers using Interquartile Range method
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - (multiplier * IQR)
    upper_bound = Q3 + (multiplier * IQR)
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    
    return outliers, lower_bound, upper_bound

print("Outlier detection function defined")

In [ ]:
# Check for outliers in key numeric columns
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in ['year', 'has_rail', 'has_port']]

print("OUTLIER ANALYSIS")
print("="*60)

outlier_summary = []
for col in numeric_cols:
    outliers, lower, upper = detect_outliers_iqr(df_clean, col)
    if len(outliers) > 0:
        outlier_summary.append({
            'Column': col,
            'Outlier_Count': len(outliers),
            'Outlier_Percent': round(len(outliers) / len(df_clean) * 100, 2),
            'Lower_Bound': round(lower, 2),
            'Upper_Bound': round(upper, 2)
        })

outlier_df = pd.DataFrame(outlier_summary).sort_values('Outlier_Percent', ascending=False)
print(outlier_df.to_string(index=False))

In [ ]:
# Visualize distributions with outliers
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.ravel()

key_cols = ['gdp_per_capita_ppp', 'co2_emissions_per_capita', 'road_density_km_per_100sqkm',
            'energy_use_per_gdp', 'lpi_overall_score', 'urban_population_pct',
            'air_transport_passengers', 'trade_pct_gdp', 'gross_capital_formation_pct_gdp']

for idx, col in enumerate(key_cols):
    if col in df_clean.columns:
        axes[idx].boxplot(df_clean[col].dropna(), vert=True)
        axes[idx].set_title(col.replace('_', ' ').title(), fontsize=10)
        axes[idx].grid(alpha=0.3)

plt.suptitle('Distribution Analysis - Box Plots', fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig('../visualizations/outlier_detection_boxplots.png', dpi=300, bbox_inches='tight')
plt.show()

print("Visualization saved: visualizations/outlier_detection_boxplots.png")

### Outlier Treatment Decision

**Decision:** Keep outliers for most indicators
- Economic diversity is real (GDP ranges from $500 to $100k)
- Infrastructure variation reflects development levels
- Extreme values are informative for clustering and analysis

**Exception:** Cap extreme values only if they appear to be data errors (we'll use log transforms in modeling instead)

## 5. Exploratory Data Analysis

### 5.1 Distribution Analysis by Income Group

In [ ]:
# Summary statistics by income group
print("GDP PER CAPITA BY INCOME GROUP")
print("="*60)
print(df_clean.groupby('income_group')['gdp_per_capita_ppp'].describe())

print("\n" + "="*60)
print("CO2 EMISSIONS PER CAPITA BY INCOME GROUP")
print("="*60)
print(df_clean.groupby('income_group')['co2_emissions_per_capita'].describe())

In [ ]:
# Visualization: Key metrics by income group
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# GDP by income group
df_clean.boxplot(column='gdp_per_capita_ppp', by='income_group', ax=axes[0,0])
axes[0,0].set_title('GDP per Capita by Income Group')
axes[0,0].set_xlabel('Income Group')
axes[0,0].set_ylabel('GDP per Capita (PPP)')

# CO2 emissions by income group
df_clean.boxplot(column='co2_emissions_per_capita', by='income_group', ax=axes[0,1])
axes[0,1].set_title('CO2 Emissions per Capita by Income Group')
axes[0,1].set_xlabel('Income Group')
axes[0,1].set_ylabel('CO2 Emissions (metric tons per capita)')

# LPI by income group
df_clean.boxplot(column='lpi_overall_score', by='income_group', ax=axes[1,0])
axes[1,0].set_title('Logistics Performance by Income Group')
axes[1,0].set_xlabel('Income Group')
axes[1,0].set_ylabel('LPI Score (1-5)')

# Urbanization by income group
df_clean.boxplot(column='urban_population_pct', by='income_group', ax=axes[1,1])
axes[1,1].set_title('Urbanization by Income Group')
axes[1,1].set_xlabel('Income Group')
axes[1,1].set_ylabel('Urban Population (%)')

plt.suptitle('')  # Remove default title
plt.tight_layout()
plt.savefig('../visualizations/income_group_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("Visualization saved: visualizations/income_group_comparison.png")

### 5.2 Temporal Trends Analysis

In [ ]:
# Calculate average trends over time by income group
trend_data = df_clean.groupby(['year', 'income_group']).agg({
    'gdp_per_capita_ppp': 'mean',
    'co2_emissions_per_capita': 'mean',
    'urban_population_pct': 'mean',
    'lpi_overall_score': 'mean'
}).reset_index()

print("Temporal trend data prepared")
print(trend_data.head())

In [ ]:
# Visualization: Trends over time
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# GDP trend
for income in df_clean['income_group'].unique():
    data = trend_data[trend_data['income_group'] == income]
    axes[0,0].plot(data['year'], data['gdp_per_capita_ppp'], marker='o', label=income)
axes[0,0].set_title('GDP per Capita Trend (2010-2022)', fontweight='bold')
axes[0,0].set_xlabel('Year')
axes[0,0].set_ylabel('GDP per Capita (PPP)')
axes[0,0].legend()
axes[0,0].grid(alpha=0.3)

# CO2 trend
for income in df_clean['income_group'].unique():
    data = trend_data[trend_data['income_group'] == income]
    axes[0,1].plot(data['year'], data['co2_emissions_per_capita'], marker='o', label=income)
axes[0,1].set_title('CO2 Emissions per Capita Trend (2010-2022)', fontweight='bold')
axes[0,1].set_xlabel('Year')
axes[0,1].set_ylabel('CO2 per Capita (metric tons)')
axes[0,1].legend()
axes[0,1].grid(alpha=0.3)

# Urbanization trend
for income in df_clean['income_group'].unique():
    data = trend_data[trend_data['income_group'] == income]
    axes[1,0].plot(data['year'], data['urban_population_pct'], marker='o', label=income)
axes[1,0].set_title('Urbanization Trend (2010-2022)', fontweight='bold')
axes[1,0].set_xlabel('Year')
axes[1,0].set_ylabel('Urban Population (%)')
axes[1,0].legend()
axes[1,0].grid(alpha=0.3)

# LPI trend
for income in df_clean['income_group'].unique():
    data = trend_data[trend_data['income_group'] == income]
    axes[1,1].plot(data['year'], data['lpi_overall_score'], marker='o', label=income)
axes[1,1].set_title('Logistics Performance Trend (2010-2022)', fontweight='bold')
axes[1,1].set_xlabel('Year')
axes[1,1].set_ylabel('LPI Score (1-5)')
axes[1,1].legend()
axes[1,1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../visualizations/temporal_trends.png', dpi=300, bbox_inches='tight')
plt.show()

print("Visualization saved: visualizations/temporal_trends.png")

### 5.3 Correlation Analysis

In [ ]:
# Select numeric columns for correlation
correlation_cols = [
    'gdp_per_capita_ppp', 'road_density_km_per_100sqkm', 'rail_lines_total_km',
    'air_transport_passengers', 'energy_use_per_gdp', 'co2_emissions_per_capita',
    'co2_transport_pct_total', 'trade_pct_gdp', 'urban_population_pct',
    'lpi_overall_score', 'lpi_infrastructure_score', 'gross_capital_formation_pct_gdp'
]

# Calculate correlation matrix
corr_matrix = df_clean[correlation_cols].corr()

print("Correlation matrix calculated")
print(f"Matrix shape: {corr_matrix.shape}")

In [ ]:
# Visualize correlation matrix
plt.figure(figsize=(14, 12))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix - Transportation & Economic Indicators', 
          fontsize=14, fontweight='bold', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('../visualizations/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("Visualization saved: visualizations/correlation_matrix.png")

In [ ]:
# Key correlations with LPI (our potential target variable)
print("\nKEY CORRELATIONS WITH LOGISTICS PERFORMANCE (LPI):")
print("="*60)
lpi_corr = corr_matrix['lpi_overall_score'].sort_values(ascending=False)
print(lpi_corr[lpi_corr.index != 'lpi_overall_score'])

### 5.4 Key Relationships - Scatter Plots

In [ ]:
# Create scatter plots for key relationships
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# GDP vs LPI
for income in df_clean['income_group'].unique():
    data = df_clean[df_clean['income_group'] == income]
    axes[0,0].scatter(data['gdp_per_capita_ppp'], data['lpi_overall_score'], 
                     alpha=0.5, label=income, s=30)
axes[0,0].set_xlabel('GDP per Capita (PPP)')
axes[0,0].set_ylabel('LPI Score')
axes[0,0].set_title('Economic Development vs Logistics Performance')
axes[0,0].legend()
axes[0,0].grid(alpha=0.3)

# GDP vs CO2
for income in df_clean['income_group'].unique():
    data = df_clean[df_clean['income_group'] == income]
    axes[0,1].scatter(data['gdp_per_capita_ppp'], data['co2_emissions_per_capita'], 
                     alpha=0.5, label=income, s=30)
axes[0,1].set_xlabel('GDP per Capita (PPP)')
axes[0,1].set_ylabel('CO2 per Capita')
axes[0,1].set_title('Economic Development vs Environmental Impact')
axes[0,1].legend()
axes[0,1].grid(alpha=0.3)

# Road Density vs LPI
for income in df_clean['income_group'].unique():
    data = df_clean[df_clean['income_group'] == income]
    axes[1,0].scatter(data['road_density_km_per_100sqkm'], data['lpi_overall_score'], 
                     alpha=0.5, label=income, s=30)
axes[1,0].set_xlabel('Road Density (km per 100 sq km)')
axes[1,0].set_ylabel('LPI Score')
axes[1,0].set_title('Infrastructure vs Logistics Performance')
axes[1,0].legend()
axes[1,0].grid(alpha=0.3)

# Urbanization vs CO2
for income in df_clean['income_group'].unique():
    data = df_clean[df_clean['income_group'] == income]
    axes[1,1].scatter(data['urban_population_pct'], data['co2_emissions_per_capita'], 
                     alpha=0.5, label=income, s=30)
axes[1,1].set_xlabel('Urban Population (%)')
axes[1,1].set_ylabel('CO2 per Capita')
axes[1,1].set_title('Urbanization vs Environmental Impact')
axes[1,1].legend()
axes[1,1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../visualizations/key_relationships.png', dpi=300, bbox_inches='tight')
plt.show()

print("Visualization saved: visualizations/key_relationships.png")

## 6. Save Cleaned Dataset

In [ ]:
# Save cleaned data
output_path = '../data/processed/transport_data_cleaned.csv'
df_clean.to_csv(output_path, index=False)

print("="*60)
print("CLEANED DATASET SAVED")
print("="*60)
print(f"File: {output_path}")
print(f"Shape: {df_clean.shape}")
print(f"Missing values: {df_clean.isnull().sum().sum()}")
print(f"\nColumns: {list(df_clean.columns)}")

## Key Findings from EDA

### 1. Income Group Patterns
- Clear stratification: High-income economies have higher GDP, emissions, and LPI scores
- Low-income economies show lower infrastructure density but rapid urbanization

### 2. Temporal Trends
- GDP growth across all income groups (2010-2022)
- Urbanization increasing, especially in middle-income economies
- CO2 emissions show mixed trends - decreasing in high-income, increasing in middle-income

### 3. Key Correlations
- Strong positive: GDP ↔ LPI (0.7+), GDP ↔ CO2 emissions
- Infrastructure correlates with logistics performance
- Energy efficiency improves with economic development

### 4. Business Implications
- Infrastructure investment drives logistics performance
- Economic development creates environmental trade-offs
- Urbanization pressure requires transportation solutions

---

**Next Steps:** Feature engineering to create efficiency metrics and predictive features